In [ ]:
import sys
print(sys.version)

In [ ]:
import os
import pickle

# input 
familyName = "eggnog"
data_directory = "11952data/"+familyName+"/"
in_tag = familyName + "_0.8"
in_parseFirstPar = True # keep first parameter or not
in_window = 1
outputPath = "output/GSA-test/" + in_tag + "/" + data_directory.split('/')[-2] + "/" # MIKE: no 'l' at the end. It is ridiculous!!

In [ ]:
# 1.Get a dict of feature profiles (out: a hkName list, a featureProfile dict)
%run FeatureTrace.ipynb
FeatTrace = FeatureTrace

traceName_li = list(filter(lambda f:f.endswith('.trace.hooklog'), os.listdir(data_directory))) # hooklog Name List

fp_dict = dict()
execTrace_dict = dict()

for f in traceName_li:
    featureTrace = FeatTrace(data_directory + f).getTrace_noContainTS()
    fp_dict[f] = featureTrace
    
    execTrace = FeatTrace(data_directory + f).getTrace_containTS()
    execTrace_dict[f] = execTrace

In [ ]:
# 2.Get a dict of pairwise alignment result 
# (in: a featureProfile dict; out: alignmentProfile dict, alingmnt baseline)

# %run Alignment3.ipynb
%run Alignment_Fast3.ipynb

BASE = traceName_li[0] # randomly pick a BASE trace log
# BASE = "008e76aafe9bd3132293816e6b0ac8e68cea5445c47065e53d7028ddbe375843_3272.trace.hooklog" # specified base
fpBASE = fp_dict[BASE]
print(BASE)
align_dict = {hk:globalAlign( fpBASE, fp_dict[hk], 10, -1, 0)[2] for hk in traceName_li } # pairwise all

In [ ]:
# 3. Get matchMatrix, gapSeqList
# (in: alignmentProfile dict; out: matchMatrix, gapSeqList)
%run StructMatchGap3.ipynb

data = structMatchGap(align_dict, BASE)
matchMatrix = data[0]
gapSeqList = data[1]

In [ ]:
# 4. Get stageMatrix 
# (in: matchMatrix, gapSeqList; out: stageMatrix)
% run StageMatrix.ipynb

stageMatrixResult = stageMatrix(matchMatrix, gapSeqList)

In [ ]:
#  5. Get Motif Elements
# (in: stageMatrixResult, BASE; out: Motif object)
% run Motif.ipynb

myMotif = Motif(stageMatrixResult, BASE)

In [ ]:
######  Garbage Code  -->>>  Work around only!   #####
###  WJ:  I don't know why do this!?

#with open('docs/label_dict-m2t-t2m-2014.pickle', 'rb') as f:
#    label_dict = pickle.load(f)[1] #(proc_list,proc_dict,label_list, label_dict)
#label_dict = {k[:-len('.trace.hooklog')]:label_dict[k] for k in label_dict}

# MIKE: 20170808, I comment it. Need to come back again.
label_dict = dict()

In [ ]:
## Drawing stages graphs
## output into 'outputPath'
% run OutputStageBarchart3.ipynb

if not os.path.isdir(outputPath): os.makedirs(outputPath)
outputStage = OutputStage( stageMatrixResult, outputPath, BASE, myMotif ) # output stages graphs
OutputMotiGraph(stageMatrixResult, BASE, fp_dict, outputStage, outputPath, myMotif, label_dict) # output motif graphs

In [ ]:
## Output common motif analysis
# output into 'outputPath'
% run CommonMotifAnalysis3.ipynb
execTrace_dict = fp_dict
motiDict = MotiDict(stageMatrixResult, myMotif, execTrace_dict, outputStage)
comMotif = motiDict.getComMoti()
# comResAnalysis = ComResAnalysis(comMotif)

In [ ]:
% run OutputCommonMotif3.ipynb
OutputCsv(outputPath, comMotif, stageMatrixResult, myMotif, BASE, execTrace_dict, outputStage)
# OutputComMotiAnaly(comResAnalysis, outputPath)